<h1 style="color:red;"><center> Cuda Docummentation </center></h1> 

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

<h1><center>Partie Code</center></h1>  





Hello Word!  :)

In [116]:
%%cu

#include<stdio.h>
#include<stdlib.h>
#include<cuda.h>

__global__ void  Hello(){
    
printf("Hello Word ! I' am GPU");

}
int main(){

Hello<<<1,1>>>();


    return 0;
}




  <h5><center>Adddition de deux nombre</center></h5>               

In [117]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda.h>
#define size sizeof(int)

__global__ void  Add(int *a, int *b,int *c){
  int i = blockIdx.x ;
    
           c[i]=a[i]+b[i];

}


int main(){
int *a,*b,*c;
int  *a_d,*b_d,*c_d;
int f = 2;
int d = 4 ;

a=(int*)malloc(size);
b=(int*)malloc(size);

*a=f;
*b=d;
c=(int*)malloc(size);
cudaMalloc((void**)&a_d,size);
cudaMalloc((void**)&b_d,size);
cudaMalloc((void**)&c_d,size);


cudaMemcpy(a_d,a,size,cudaMemcpyHostToDevice);
cudaMemcpy(b_d,b,size,cudaMemcpyHostToDevice);

Add<<<1,1>>>(a_d,b_d,c_d);
cudaMemcpy(c,c_d,size,cudaMemcpyDeviceToHost);



printf("la somme  des deux Nombre calculer par GPU est %d",*c);

    cudaFree(a_d);
    cudaFree(b_d);
    cudaFree(c_d);


    return 0;
}

la somme  des deux Nombre calculer par GPU est 6


   <h5><center> Addition de deux vecteur avec N block et 1 Thread</center></h5>   

In [119]:
%%cu
#include<stdio.h>
#include <malloc.h>
#include<cuda.h>
#include<stdlib.h>
#define N 512
__global__ void ad( int *a, int *b, int *c ) { 
    

        int i = blockIdx.x * blockDim.x + threadIdx.x;
    
           c[i]=a[i]+b[i];

    
 
           
}

int main(){

int a[N],b[N],c[N];
int *A_a,*B_b,*C_c;
int size = N * sizeof(int);

printf("\n  elements of first array :\n");
    for(int i=0;i<N;i++)
    {   
        
       a[i]=rand()%100;;
     printf("%d\t",a[i]);
    }
    printf("\n  elements of second array :\n");
        for(int i=0;i<N;i++)
        {
            b[i]=rand()%100;
         printf("%d\t",b[i]);
        }


printf("lLes Variables de device GPU");

printf("allocation du mamoire de GPU");
cudaMalloc(&A_a,size);
cudaMalloc(&B_b,size);
cudaMalloc(&C_c,size);


printf("copy data de CPU ver GPU ");
cudaMemcpy(A_a,&a, size, cudaMemcpyHostToDevice);
cudaMemcpy(B_b,&b, size, cudaMemcpyHostToDevice);
cudaMemcpy(C_c,&c, size, cudaMemcpyHostToDevice);




printf("exeucion sur gpu de fonction add");
ad <<< N , 1 >>> ( A_a , B_b , C_c );


cudaMemcpy (&c ,C_c, size , cudaMemcpyDeviceToHost ) ;
printf("le resultat est :\n");
for(int i=0;i<N;i++){
printf("%d\t",c[i]);}





    return 0;
}


  elements of first array :
83	86	77	15	93	35	86	92	49	21	62	27	90	59	63	26	40	26	72	36	11	68	67	29	82	30	62	23	67	35	29	2	22	58	69	67	93	56	11	42	29	73	21	19	84	37	98	24	15	70	13	26	91	80	56	73	62	70	96	81	5	25	84	27	36	5	46	29	13	57	24	95	82	45	14	67	34	64	43	50	87	8	76	78	88	84	3	51	54	99	32	60	76	68	39	12	26	86	94	39	95	70	34	78	67	1	97	2	17	92	52	56	1	80	86	41	65	89	44	19	40	29	31	17	97	71	81	75	9	27	67	56	97	53	86	65	6	83	19	24	28	71	32	29	3	19	70	68	8	15	40	49	96	23	18	45	46	51	21	55	79	88	64	28	41	50	93	0	34	64	24	14	87	56	43	91	27	65	59	36	32	51	37	28	75	7	74	21	58	95	29	37	35	93	18	28	43	11	28	29	76	4	43	63	13	38	6	40	4	18	28	88	69	17	17	96	24	43	70	83	90	99	72	25	44	90	5	39	54	86	69	82	42	64	97	7	55	4	48	11	22	28	99	43	46	68	40	22	11	10	5	1	61	30	78	5	20	36	44	26	22	65	8	16	82	58	24	37	62	24	0	36	52	99	79	50	68	71	73	31	81	30	33	94	60	63	99	81	99	96	59	73	13	68	90	95	26	66	84	40	90	84	76	42	36	7	45	56	79	18	87	12	48	72	59	9	36	10	42	87	6	1	13	72	21	55	19	99	21	4	39	11	40	67

<h5><center>1 block   avec  N thread</center></h5>   
   
   


In [ ]:
%%cu
#include<stdio.h>
#include <malloc.h>
#include<cuda.h>
#include<stdlib.h>
#define N 512
__global__ void ad( int *a, int *b, int *c ) { 
    

        int i = blockIdx.x * blockDim.x + threadIdx.x;
    
           c[i]=a[i]+b[i];

    
 
           
}

int main(){

int a[N],b[N],c[N];
int *A_a,*B_b,*C_c;
int size = N * sizeof(int);

printf("\n  elements of first array :\n");
    for(int i=0;i<N;i++)
    {   
        
       a[i]=rand()%100;;
     printf("%d\t",a[i]);
    }
    printf("\n  elements of second array :\n");
        for(int i=0;i<N;i++)
        {
            b[i]=rand()%100;
         printf("%d\t",b[i]);
        }


printf("lLes Variables de device GPU");

printf("allocation du mamoire de GPU");
cudaMalloc(&A_a,size);
cudaMalloc(&B_b,size);
cudaMalloc(&C_c,size);


printf("copy data de CPU ver GPU ");
cudaMemcpy(A_a,&a, size, cudaMemcpyHostToDevice);
cudaMemcpy(B_b,&b, size, cudaMemcpyHostToDevice);
cudaMemcpy(C_c,&c, size, cudaMemcpyHostToDevice);




printf("exeucion sur gpu de fonction add");
ad <<< 1 , N>>> ( A_a , B_b , C_c );


cudaMemcpy (&c ,C_c, size , cudaMemcpyDeviceToHost ) ;
printf("le resultat est :\n");
for(int i=0;i<N;i++){
printf("%d\t",c[i]);}





    return 0;
}

<h5><center>8 block  64 Thread</center></h5> 

In [ ]:
%%cu
#include<stdio.h>
#include <malloc.h>
#include<cuda.h>
#include<stdlib.h>
#define N 512
__global__ void ad( int *a, int *b, int *c ) { 
    

        int i = blockIdx.x * blockDim.x + threadIdx.x;
    
           c[i]=a[i]+b[i];

    
 
           
}

int main(){

int a[N],b[N],c[N];
int *A_a,*B_b,*C_c;
int size = N * sizeof(int);

printf("\n  elements of first array :\n");
    for(int i=0;i<N;i++)
    {   
        
       a[i]=rand()%100;;
     printf("%d\t",a[i]);
    }
    printf("\n  elements of second array :\n");
        for(int i=0;i<N;i++)
        {
            b[i]=rand()%100;
         printf("%d\t",b[i]);
        }


printf("lLes Variables de device GPU");

printf("allocation du mamoire de GPU");
cudaMalloc(&A_a,size);
cudaMalloc(&B_b,size);
cudaMalloc(&C_c,size);


printf("copy data de CPU ver GPU ");
cudaMemcpy(A_a,&a, size, cudaMemcpyHostToDevice);
cudaMemcpy(B_b,&b, size, cudaMemcpyHostToDevice);
cudaMemcpy(C_c,&c, size, cudaMemcpyHostToDevice);




printf("exeucion sur gpu de fonction add");
ad <<< 8 , N/8>>> ( A_a , B_b , C_c );


cudaMemcpy (&c ,C_c, size , cudaMemcpyDeviceToHost ) ;
printf("le resultat est :\n");
for(int i=0;i<N;i++){
printf("%d\t",c[i]);}





    return 0;
}


<h5><center>Temps d'execution  avec N bloc et 1 thread</center></h5>   







In [122]:
!nvcc /content/src/Add_vec_Nbloc_1Thread.cu -o A
!nvprof ./A


  elements of first array :
83	86	77	15	93	35	86	92	49	21	62	27	90	59	63	26	40	26	72	36	11	68	67	29	82	30	62	23	67	35	29	2	22	58	69	67	93	56	11	42	29	73	21	19	84	37	98	24	15	70	13	26	91	80	56	73	62	70	96	81	5	25	84	27	36	5	46	29	13	57	24	95	82	45	14	67	34	64	43	50	87	8	76	78	88	84	3	51	54	99	32	60	76	68	39	12	26	86	94	39	95	70	34	78	67	1	97	2	17	92	52	56	1	80	86	41	65	89	44	19	40	29	31	17	97	71	81	75	9	27	67	56	97	53	86	65	6	83	19	24	28	71	32	29	3	19	70	68	8	15	40	49	96	23	18	45	46	51	21	55	79	88	64	28	41	50	93	0	34	64	24	14	87	56	43	91	27	65	59	36	32	51	37	28	75	7	74	21	58	95	29	37	35	93	18	28	43	11	28	29	76	4	43	63	13	38	6	40	4	18	28	88	69	17	17	96	24	43	70	83	90	99	72	25	44	90	5	39	54	86	69	82	42	64	97	7	55	4	48	11	22	28	99	43	46	68	40	22	11	10	5	1	61	30	78	5	20	36	44	26	22	65	8	16	82	58	24	37	62	24	0	36	52	99	79	50	68	71	73	31	81	30	33	94	60	63	99	81	99	96	59	73	13	68	90	95	26	66	84	40	90	84	76	42	36	7	45	56	79	18	87	12	48	72	59	9	36	10	42	87	6	1	13	72	21	55	19	99	21	4	39	11	40	67

<h5><center> Temps d'execution  avec 1 bloc et N thread
</center></h5>     


In [123]:
!nvcc /content/src/Add_vec1bloc_N_thread.cu -o A
!nvprof ./A


  elements of first array :
83	86	77	15	93	35	86	92	49	21	62	27	90	59	63	26	40	26	72	36	11	68	67	29	82	30	62	23	67	35	29	2	22	58	69	67	93	56	11	42	29	73	21	19	84	37	98	24	15	70	13	26	91	80	56	73	62	70	96	81	5	25	84	27	36	5	46	29	13	57	24	95	82	45	14	67	34	64	43	50	87	8	76	78	88	84	3	51	54	99	32	60	76	68	39	12	26	86	94	39	95	70	34	78	67	1	97	2	17	92	52	56	1	80	86	41	65	89	44	19	40	29	31	17	97	71	81	75	9	27	67	56	97	53	86	65	6	83	19	24	28	71	32	29	3	19	70	68	8	15	40	49	96	23	18	45	46	51	21	55	79	88	64	28	41	50	93	0	34	64	24	14	87	56	43	91	27	65	59	36	32	51	37	28	75	7	74	21	58	95	29	37	35	93	18	28	43	11	28	29	76	4	43	63	13	38	6	40	4	18	28	88	69	17	17	96	24	43	70	83	90	99	72	25	44	90	5	39	54	86	69	82	42	64	97	7	55	4	48	11	22	28	99	43	46	68	40	22	11	10	5	1	61	30	78	5	20	36	44	26	22	65	8	16	82	58	24	37	62	24	0	36	52	99	79	50	68	71	73	31	81	30	33	94	60	63	99	81	99	96	59	73	13	68	90	95	26	66	84	40	90	84	76	42	36	7	45	56	79	18	87	12	48	72	59	9	36	10	42	87	6	1	13	72	21	55	19	99	21	4	39	11	40	67

 ### Temps d'execution 8 blok 64 Thread`
  
 


In [124]:
!nvcc /content/src/Add_vec_8block_64Thread.cu -o A
!nvprof ./A

/content/src/Add_vec_8block_64Thread.cu(1): error: expected a declaration

/usr/include/malloc.h(163): error: expected a declaration

2 errors detected in the compilation of "/tmp/tmpxft_00005a27_00000000-8_Add_vec_8block_64Thread.cpp1.ii".

  elements of first array :
83	86	77	15	93	35	86	92	49	21	62	27	90	59	63	26	40	26	72	36	11	68	67	29	82	30	62	23	67	35	29	2	22	58	69	67	93	56	11	42	29	73	21	19	84	37	98	24	15	70	13	26	91	80	56	73	62	70	96	81	5	25	84	27	36	5	46	29	13	57	24	95	82	45	14	67	34	64	43	50	87	8	76	78	88	84	3	51	54	99	32	60	76	68	39	12	26	86	94	39	95	70	34	78	67	1	97	2	17	92	52	56	1	80	86	41	65	89	44	19	40	29	31	17	97	71	81	75	9	27	67	56	97	53	86	65	6	83	19	24	28	71	32	29	3	19	70	68	8	15	40	49	96	23	18	45	46	51	21	55	79	88	64	28	41	50	93	0	34	64	24	14	87	56	43	91	27	65	59	36	32	51	37	28	75	7	74	21	58	95	29	37	35	93	18	28	43	11	28	29	76	4	43	63	13	38	6	40	4	18	28	88	69	17	17	96	24	43	70	83	90	99	72	25	44	90	5	39	54	86	69	82	42	64	97	7	55	4	48	11	22	28	99	43	46	68	40	22	11	10	

 ### produit scalaire

In [125]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include <stdlib.h> 
#define N 512
__global__ void dot(int *a , int *b , int *c){


    __shared__ int temp[N];

    temp[blockDim.x * blockIdx.x + threadIdx.x] = a[blockDim.x * blockIdx.x + threadIdx.x] * b[blockDim.x * blockIdx.x + threadIdx.x ] ;

     __syncthreads();

    if(0==threadIdx.x){
          int sum = 0;
          for(int i = 0; i < N ; i++)
                sum = sum +temp[i];

                *c = sum ;
    }
}




int main (){
 int *a , *b, *c;
 int *d_a , *d_b ,*d_c ;
 int size = N * sizeof (int);


 cudaMalloc((void**)&d_a,size);
 cudaMalloc((void**)&d_b,size);
 cudaMalloc((void**)&d_c,size);

 a=(int *)malloc(size);
 b=(int *)malloc(size);
 c=(int *)malloc(size);  

 
    for(int i=0;i<N;i++)
    {   
        
       a[i]=1;
    
    }
   
        for(int i=0;i<N;i++)
        {
            b[i]=1;
        }




cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
cudaMemcpy(d_b,b,size,cudaMemcpyHostToDevice);

dot<<<1,N>>>(d_a,d_b,d_c);
 
cudaMemcpy(c,d_c,size,cudaMemcpyDeviceToHost);

 printf("Le produit scalaire est : %d \n", *c) ;

free(a);free(b);free(c);

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

return 0;
}

Le produit scalaire est : 512 



### temps d'execution  de produit Sclaire

In [126]:
!nvcc /content/src/produit_scalaire.cu -o A
!nvprof ./A

==23180== NVPROF is profiling process 23180, command: ./A
==23180== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==23180== Profiling application: ./A
==23180== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.81%  12.864us         1  12.864us  12.864us  12.864us  dot(int*, int*, int*)
                   22.22%  4.4800us         2  2.2400us  1.9840us  2.4960us  [CUDA memcpy HtoD]
                   13.97%  2.8160us         1  2.8160us  2.8160us  2.8160us  [CUDA memcpy DtoH]
      API calls:   99.36%  192.10ms         3  64.034ms  1.9560us  192.09ms  cudaMalloc
                    0.27%  527.92us         1  527.92us  527.92us  527.92us  cuDeviceTotalMem
                    0.13%  248.93us  